# Integration of lakeFS with Airflow via Hooks

## Use Case: Versioning DAGs and running pipeline from hooks using a configurable version of DAGs

## Versioning Information

In [ ]:
sourceBranch = "main"
newBranch = "version1"
fileName = "lakefs_test.csv"
local_path = 'airflow/DAG_Versioning'
dags_folder_on_lakefs = 'dags'
data_folder_on_lakefs = 'data'
actions_folder_on_lakefs = '_lakefs_actions'
newPath = data_folder_on_lakefs + '/' + "partitioned_data"
dag_name = 'lakefs_versioning_dag'
dag_template_filename = 'lakefs_versioning_dag_template.py'
dags_folder_on_github = 'https://raw.githubusercontent.com/treeverse/lakeFS-samples/main/03-multiple-samples/airflow/DAG_Versioning'

## Run [Common Setup](../Common/CommonSetup.ipynb) tasks here

In [ ]:
%run ./airflow/Common/CommonSetup.ipynb

## Import Python packages

In [ ]:
# importing sys
import sys
 
# adding folder to the system path
sys.path.insert(0, './'+local_path)
 
from lakefs_create_dag import lakefs_create_dag

## Create Airflow connections for lakeFS

In [ ]:
! airflow connections delete conn_http_github_dag_template
httpConnectionCommand = 'airflow connections add conn_http_github_dag_template --conn-type=http --conn-host=' + dags_folder_on_github + '/' +dag_template_filename
! $httpConnectionCommand > ./airflow/airflow-connection.txt
with open("./airflow/airflow-connection.txt", "r") as file:
    last_line = file.readlines()[-1]
print(last_line)

## Set Airflow variables which are used by the demo workflow

In [ ]:
fileName_on_lakefs = data_folder_on_lakefs + '/' + fileName
! airflow variables set fileName $fileName_on_lakefs
! airflow variables set newPath $newPath
! airflow variables set dags_folder_on_lakefs $dags_folder_on_lakefs
! airflow variables set dag_name $dag_name
! airflow variables set dag_template_filename $dag_template_filename
! airflow variables set dags_folder_on_github $dags_folder_on_github

## Copy DAG programs to Airflow DAGs directory and sync to Airflow database

In [ ]:
! cp ./airflow/DAG_Versioning/lakefs_create_dag_auto.py ./airflow/dags
! cp ./airflow/DAG_Versioning/lakefs_delete_dag_auto.py ./airflow/dags
! cp ./airflow/DAG_Versioning/lakefs_trigger_dag_auto.py ./airflow/dags

dagbag = DagBag(include_examples=False)
dagbag.sync_to_db()

## Unpause Airflow DAGs

In [ ]:
! airflow dags unpause lakefs_create_dag
! airflow dags unpause lakefs_delete_dag
! airflow dags unpause lakefs_trigger_dag